In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px  # For Plotly Express, which is easy-to-use for quick visualizations
import plotly.graph_objects as go 

In [ ]:
from functions import concatenate_eurostat_countries, cleaning_eurostat_data_for_viz, clean_countries_numbeo, clean_cities
eurostat_df = concatenate_eurostat_countries()
eurostat_df

In [ ]:
cities = clean_cities()
cities

In [ ]:
countries = clean_countries_numbeo()
countries

In [ ]:
def cleaning_eurostat_data_for_viz(eurostat_df):
    income_df = eurostat_df[eurostat_df.index == 'Income']
    housing_df = eurostat_df[eurostat_df.index == "Housing"]
    rental_df = eurostat_df[eurostat_df.index == "Rental"]

    income_tidy = income_df.melt(id_vars="Country", var_name="Year", value_name="Income")
    housing_tidy = housing_df.melt(id_vars="Country", var_name="Year", value_name="Housing")
    rental_tidy = rental_df.melt(id_vars="Country", var_name="Rental")

    return income_tidy, housing_tidy, rental_tidy, income_df, housing_df, rental_df

In [ ]:
income_tidy, housing_tidy, rental_tidy, income_df, housing_df, rental_df = cleaning_eurostat_data_for_viz(eurostat_df)

# Concatenate the DataFrames into one long DataFrame
combined_data = pd.concat([housing_df, rental_df], axis=0)
df_portugal = combined_data[combined_data['Country'] == 'PT']
df_portugal = df_portugal.reset_index().rename(columns={'index': 'Type'})
df_portugal_melted = df_portugal.melt(id_vars=['Type', 'Country'], var_name='Year', value_name='Value')
df_portugal_melted

In [ ]:
fig = px.line(
    df_portugal_melted,
    x='Year',
    y='Value',
    color='Type',  # Different lines for Income and Rental
    markers=True,
    title="Income vs Rental in Portugal (2019-2023)"
)

# Show the plot
fig.show()

In [ ]:
plt.figure(figsize=(9, 6))
sns.lineplot(data=income_tidy, x="Year", y="Income", hue="Country", marker="o")
plt.title("Income Trends for Portugal (2019-2023)")
plt.xlabel("Year")
plt.ylabel("Income (€)")
plt.show()

In [ ]:
df_lisbon = cities[cities["City"] == "Lisbon"]
df_lisbon

In [ ]:
df_rent_salary = df_lisbon[df_lisbon['Type'].isin(['1 bed apartment (rent)', '3 bed apartment (rent)', 'Av salary (after tax)'])]
df_rent_salary_melted = df_rent_salary.melt(id_vars=["Type", "City"], var_name="Year", value_name="Value")
fig = px.line(
    df_rent_salary_melted,
    x="Year",              # x-axis: Year
    y="Value",             # y-axis: Value
    color="Type",          # Different lines for different Types (rent and salary)
    markers=True,          # Add markers at each data point
    title="Comparison between Rent and Salary in Lisbon (2019-2024)", # Title of the chart
    labels={"Value": "Amount (€)", "Year": "Year", "Type": "Type of Data"} # Axis labels
)

# Show the plot
fig.show()

In [ ]:
cities_one_bed_apt = cities[cities['Type'] == "1 bed apartment (rent)"]
cities_one_bed_apt_melted = cities_one_bed_apt.melt(id_vars=["Type", "City"], var_name="Year", value_name="Value")
cities_one_bed_apt_melted

In [ ]:
fig = px.line(
    cities_one_bed_apt_melted, 
    x="Year", 
    y="Value",
    color="City",       
    line_group="City",  
    facet_col="Type",      
    markers=True 
)

fig.show()

In [ ]:
cities_salaries = cities[cities['Type'] == "Av salary (after tax)"]
cities_salaries_melted = cities_salaries.melt(id_vars=["Type", "City"], var_name="Year", value_name="Value")
cities_salaries_melted

In [ ]:
fig = px.line(
    cities_salaries_melted, 
    x="Year", 
    y="Value",
    color="City",       
    line_group="City",  
    facet_col="Type",      
    markers=True
    title="Average salaries in Lisbon, Berlin and Paris (2019-2024)", # Title of the chart
    labels={"Value": "Amount (€)", "Year": "Year", "Type": "Type of Data"} # Axis labels
)

fig.show()